In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import zipfile
import os
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from time import sleep
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import requests
from os import listdir
from os.path import isfile, join
import urllib
from urllib.request import urlretrieve 
from sentence_transformers import SentenceTransformer, util

import zipfile

from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.model_selection import ParameterGrid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\marcj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
path = 'data/'

In [3]:
df = pd.read_csv('data/20ng.csv')
df

,Text,Class
0,subject manner similar termin graphic graphic ...,comp_windows_x
1,standard cost offic subject name count bruce v...,comp_windows_x
2,subject nasa nasa nasa nasa outland voic toolk...,comp_windows_x
3,info subject server share friend friend close ...,comp_windows_x
4,advantag helen helen code support subject citi...,comp_windows_x
...,...,...
18823,investor ga ga presid vo standard subject enfo...,talk_politic_guns
18824,analysi ga ga ga ga ga laughabl beryllium bery...,talk_politic_guns
18825,na info quot illinoi illinoi illinoi illinoi s...,talk_politic_guns
18826,sacrific cold subject ccwf ccwf mikei mikei ut...,talk_politic_guns


In [4]:
def aplicar_tfidf_em_dados(dados):
    # Extrair as colunas de texto, classes e tokens
    texto = dados['text']
    classes = dados['Class']
    tokens = dados['tokens']

    # Inicializar o TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Aplicar o vetorizador ao texto
    tfidf_matrix = tfidf_vectorizer.fit_transform(texto)

    # Transformar as classes em valores numéricos usando LabelEncoder
    label_encoder = LabelEncoder()
    classes_encoded = label_encoder.fit_transform(classes)

    # Criar um DataFrame para armazenar os resultados
    resultado = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

    # Adicionar coluna de classes ao DataFrame de resultados
    resultado['classes'] = classes_encoded

    return resultado

In [5]:
x_train, x_test, y_train, y_test = train_test_split(df['Text'], df['Class'], test_size=0.2, random_state=5)

In [6]:
tfidf_vec = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vec.fit_transform(x_train)
tfidf_test = tfidf_vec.transform(x_test)

In [7]:
clf = MultinomialNB()
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

accuracy:   0.889
[[134   0   0   0   0   0   0   0   0   0   0   1   0   0   0  17   0   1
    0   0]
 [  0 165  10  14   4   5   1   0   0   1   0   4   0   1   2   0   1   0
    0   0]
 [  0   2 165  20   2   8   2   2   0   0   0   3   0   0   3   1   0   0
    0   0]
 [  0   3   7 166  12   0   4   0   1   0   0   1   3   0   0   0   0   0
    0   0]
 [  0   2   2   7 159   0   2   0   1   0   0   1   3   0   1   0   0   0
    0   0]
 [  0   5   6   3   1 173   1   0   0   1   1   1   0   0   4   0   0   1
    0   0]
 [  0   0   1  10   3   1 155   7   3   0   0   0   2   0   0   0   1   0
    1   0]
 [  0   0   0   0   0   1   2 171   5   1   1   0   1   1   0   0   3   0
    0   0]
 [  0   0   0   0   0   0   1   1 178   0   0   1   0   0   1   0   0   0
    0   0]
 [  0   0   0   0   0   0   1   1   0 184   8   0   0   0   0   2   0   0
    0   0]
 [  0   0   1   0   0   0   0   0   0   1 205   0   0   0   1   0   1   0
    0   0]
 [  0   1   1   0   0   2   0   0   0   0   0 1

In [8]:
print(classification_report(y_test, pred))

                          precision    recall  f1-score   support

             alt_atheism       0.86      0.88      0.87       153
           comp_graphics       0.92      0.79      0.85       208
 comp_os_ms-windows_misc       0.85      0.79      0.82       208
comp_sys_ibm_pc_hardware       0.72      0.84      0.78       197
   comp_sys_mac_hardware       0.87      0.89      0.88       178
          comp_windows_x       0.91      0.88      0.89       197
            misc_forsale       0.91      0.84      0.88       184
               rec_autos       0.93      0.92      0.92       186
         rec_motorcycles       0.93      0.98      0.95       182
      rec_sport_baseball       0.97      0.94      0.95       196
        rec_sport_hockey       0.94      0.98      0.96       209
               sci_crypt       0.89      0.97      0.93       201
         sci_electronics       0.94      0.86      0.90       191
                 sci_med       0.99      0.97      0.98       206
         

In [9]:
text = "i need a new gun"
text = [text]

text_tfidf = tfidf_vec.transform(text)
clf.predict(text_tfidf)

array(['talk_politic_guns'], dtype='<U24')

In [12]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import zipfile
import os
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from time import sleep
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import requests
from os import listdir
from os.path import isfile, join
import urllib
from urllib.request import urlretrieve 
from sentence_transformers import SentenceTransformer, util

import zipfile

from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.model_selection import ParameterGrid

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def train(df, model):
    x_train, x_test, y_train, y_test = train_test_split(df['Text'], df['Class'], test_size=0.2, random_state=5)
    tfidf_vec = TfidfVectorizer(stop_words='english', max_df=0.7)
    tfidf_train = tfidf_vec.fit_transform(x_train)
    tfidf_test = tfidf_vec.transform(x_test)
    if model == 'naive_bayes':
        clf = MultinomialNB()
        clf.fit(tfidf_train, y_train)
        pred = clf.predict(tfidf_test)

    elif model == 'knn':
        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(tfidf_train, y_train)
        pred = knn.predict(tfidf_test)

    score = accuracy_score(y_test, pred)
        
    return ("accuracy:   %0.3f" % score)


def main():
    list_models = ['naive_bayes', 'knn']
    df = pd.read_parquet('data/df.parquet')
    for model in list_models:
        print(model)
        print(train(df, model))
        print('-------------------------')
        
        
if __name__ == '__main__':  
    main()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


naive_bayes
accuracy:   0.391
-------------------------
knn
accuracy:   0.446
-------------------------
